In [5]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [6]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [7]:
feedback_bits = 256
ncodebooks=128
ncentroids=16
train_sam_num = 3000 # 训练集样本数
split = 16 # A的列、B的行分割成split份

In [8]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")

dir_train_split = os.path.join(dir_train, 'train', 'f'+str(feedback_bits), 'split'+str(split))
try:
    os.mkdir(dir_train_split)
except FileExistsError:
    pass
# print(dir_train_split)
dir_test_split = os.path.join(dir_train, 'test', 'f'+str(feedback_bits), 'split'+str(split))
try:
    os.mkdir(dir_test_split)
except FileExistsError:
    pass 


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



切分训练集

In [9]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_train_split_path_list = []
for i in range(split):
    data_to_fc_train_split_path_list.append(os.path.join(dir_train_split, 'data_to_fc_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_train_split_path_list[i], data_to_fc_train[np.ix_(range(data_to_fc_train.shape[0]), range(i*split_size,(i+1)*split_size))])

(96000, 2048)


In [10]:
weight_train = np.load(os.path.join(dir_train, weightpath))
print(weight_train.shape)
weight_train_split_path_list = []
for i in range(split):
    weight_train_split_path_list.append(os.path.join(dir_train_split, 'weight_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(weight_train_split_path_list[i], weight_train[np.ix_(range(i*split_size,(i+1)*split_size), range(weight_train.shape[1]))])

(2048, 256)


In [11]:
# 输入与weight相乘
y_train_split_path_list = []
for i in range(split):
    y_train_split_path_list.append(os.path.join(dir_train_split, 'y_train_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_train_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_train_split_path_list[i], np.matmul(xx, ww))

切分测试集

In [12]:
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
# split_size = int(data_to_fc_train.shape[1] / split) # split后单个训练集A的列数
data_to_fc_test_split_path_list = []
for i in range(split):
    data_to_fc_test_split_path_list.append(os.path.join(dir_test_split, 'data_to_fc_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    np.save(data_to_fc_test_split_path_list[i], data_to_fc_test[np.ix_(range(data_to_fc_test.shape[0]), range(i*split_size,(i+1)*split_size))])

(32000, 2048)


In [13]:
# 输入与weight相乘
y_test_split_path_list = []
for i in range(split):
    y_test_split_path_list.append(os.path.join(dir_test_split, 'y_test_f'+str(feedback_bits)+'_split'+str(split)+'_'+str(i)+'.npy'))
    xx = np.load(data_to_fc_test_split_path_list[i])
    ww = np.load(weight_train_split_path_list[i])
    np.save(y_test_split_path_list[i], np.matmul(xx, ww))

In [14]:
est_list = []
for i in range(split):
    dir_est, X_path = os.path.split(data_to_fc_train_split_path_list[i])
    dir_est, W_path = os.path.split(weight_train_split_path_list[i])
    dir_est, Y_path = os.path.split(y_train_split_path_list[i])
    est3 = mm.estFactory(X_path=X_path, W_path=W_path, Y_path=Y_path, dir= dir_est, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])
    est_list.append(est3)


running method:  Mithral
X_res mse / X mse:  0.016641002
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.012994092
running method:  Mithral
X_res mse / X mse:  0.0152959125
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.012704283
running method:  Mithral
X_res mse / X mse:  0.015475823
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.013594396
running method:  Mithral
X_res mse / X mse:  0.015384653
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.01340051
running method:  Mithral
X_res mse / X mse:  0.016944893
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.013002489
running method:  Mithral
X_res mse / X mse:  0.015139081
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.012707036
running method:  Mithral
X_res mse / X mse:  0.015480162
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.013525557
running method:  Mithral
X_res mse / X mse:  0.015347608
fitting dense lstsq to X_res
X_re

In [15]:
y_split_out_matmul_list = []
for i in range(split):
    x_test = np.load(data_to_fc_test_split_path_list[i])
    w_test = np.load(weight_train_split_path_list[i])
    y_split_out_matmul = mm.eval_matmul(est_list[i], x_test, w_test)
    y_split_out_matmul_list.append(y_split_out_matmul)

In [16]:
# split 的矩阵结果合成
y_out_matmul = y_split_out_matmul_list[0]
if split>1:
    for i in range(1, split):
        y_out_matmul += y_split_out_matmul_list[i]

In [17]:
bias = np.load(dir_train+'/'+biaspath)
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [18]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_split%i_fb%i_cb%i_ct%i.npy' % (split, feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[-0.23255426 -0.19161709 -0.62854695 ... -0.10785938  0.10200073
  -0.09245922]
 [-0.2756071  -0.01940576 -0.34674653 ... -0.39357367  0.03546455
   0.00147425]
 [ 0.5541385  -0.01549185 -0.17453521 ... -0.64014894  0.16853695
   0.06801046]
 ...
 [ 0.4132383   0.22325568 -0.5072162  ... -0.19005117 -0.3441832
   0.01712981]
 [ 0.29190758 -0.08202806 -0.27629647 ...  0.04086862  0.04720623
   0.18934117]
 [-0.52609634  0.03930264 -0.46024948 ... -0.307468   -0.15631628
   0.17368561]]
(32000, 256)
